# Interactive bot demo using LLMs and MLRun

In [ ]:
import os
import mlrun
import openai

# Copy mlrun.env and populate with relevant credentials
ENV_FILE = "mlrun.env"

mlrun.set_env_from_file(ENV_FILE)
project = mlrun.get_or_create_project("llamagpt")

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

### Index Documents

In [ ]:

from src import LlmIndex

llmi = LlmIndex(additional_tools=["llm-math"])

llmi.add_index(
    index_name="llama",
    documents=llmi.load_documents(documents_dir="data/text"),
    summary="useful for questions about llamas"
)
llmi.add_index(
    index_name="strawberries",
    documents=llmi.load_documents(loader="WikipediaReader", pages=['Strawberry']),
    summary="useful for questions about strawberries"
)

llmi.add_index(
    index_name="mlops",
    documents=llmi.load_documents(
        loader="BeautifulSoupWebReader",
        urls=['https://www.iguazio.com/blog/concept-drift-deep-dive-how-to-build-a-drift-aware-ml-system/']
        ),
    summary="useful for questions about mlops, machine learning, or concept drift"
)


In [ ]:

llmi.persist(persist_dir="./storage")

In [ ]:
llmi.persist(persist_dir="v3io:///bigdata/storage")

### Query Documents

In [ ]:
# load the docs & indexes from storage and run queries
llmi = LlmIndex(additional_tools=["llm-math"])
llmi.load_storage()
llmi.get_catalog()

In [ ]:
print(llmi.query("How tall are llamas?"))
print(llmi.query("Which countries produce strawberries?"))
print(llmi.query("What is MLOps?"))
print(llmi.query("What 5 times the average weight of a llama?"))

### Serving Graph

In [ ]:
fn = mlrun.code_to_function(
    name="llama",
    filename="src/llmindex.py",
    kind="serving",
    image="mlrun/mlrun:1.3.2",
    requirements=["llama-index==0.6.26", "langchain==0.0.200"]
)
graph = fn.set_topology("flow", engine="async")
graph.add_step(name="llama", class_name="QueryLlamaGpt", persist_dir="v3io:///bigdata/storage", additional_tools=["llm-math"], full_event=True).respond()
fn.set_env("OPENAI_API_KEY", os.getenv("OPENAI_API_KEY"))
fn.set_env("OPENAI_API_BASE", os.getenv("OPENAI_API_BASE"))

project.deploy_function(fn, mock=True)

In [ ]:
query = "How tall are llamas?"
resp = fn.invoke(path="query", body={"query" : query})
resp["response"]

In [ ]:
query = "What is 50 times the height of the tallest llama in the world?"
resp = fn.invoke(path="query", body={"query" : query})
resp["response"]

### Gradio App

In [ ]:
import gradio as gr

In [ ]:
def query_llama(query: str) -> str:
    resp = fn.invoke(path="/query", body={"query" : query})
    query_result = resp["response"]
    return query_result

with gr.Blocks(analytics_enabled=False, theme=gr.themes.Soft()) as demo:
    with gr.Row():
        chatbot = gr.Chatbot()
    with gr.Row():
        with gr.Column(scale=6):
            msg = gr.Textbox(label="Q:", placeholder="Type a question and Enter")
        with gr.Column(scale=3):
            clear = gr.Button("Clear")

    def respond(message, chat_history, index):
        bot_message = query_llama(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()